# Data cleaning

In [2]:
import sys  
!{sys.executable} -m pip install --user pycox

  Using cached pycox-0.2.2-py3-none-any.whl (73 kB)
  Using cached feather-format-0.4.1.tar.gz (3.2 kB)
  Using cached py7zr-0.16.1-py3-none-any.whl (65 kB)
     |████████████████████████████████| 17.6 MB 1.4 MB/s eta 0:00:01
  Using cached pycryptodomex-3.10.1-cp35-abi3-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached multivolumefile-0.2.3-py3-none-any.whl (17 kB)
  Using cached texttable-1.6.4-py2.py3-none-any.whl (10 kB)
     |████████████████████████████████| 432 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 421 kB 1.2 MB/s eta 0:00:01
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2454 sha256=a302c0ce9f65fab7f2058393f8cb7a89a3bf276b4b9174d5463633ed9a3b6a84
  Stored in directory: /Users/camillacallierotti/Library/Caches/pip/wheels/7b/83/4b/44aa8f2292c0ae17a550c5105454e974fc45a5b2a522d66ae1
Successfully built feather-format
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-scrip

In [87]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import torch
import torchtuples as tt

from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

## Dataset

In [90]:
# import dataset

url = 'https://raw.githubusercontent.com/camicallierotti/imperial-summer-project/main/pbc.csv'
df = pd.read_csv(url, sep=";", encoding='latin1',engine='python', header=0, decimal=',')
df.head(5)

,id,time,status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
0,1,400,2,D-penicillmain,58.765229,female,yes,yes,yes,edema,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,IV
1,2,4500,0,D-penicillmain,56.446270,female,no,yes,yes,no,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,III
2,3,1012,2,D-penicillmain,70.072553,male,no,no,no,untreated,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,IV
3,4,1925,2,D-penicillmain,54.740589,female,no,yes,yes,untreated,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,IV
4,5,1504,1,Placebo,38.105407,female,no,yes,yes,no,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,III


In [92]:
# label encoding

df = df.applymap(str) # converto whole df to strings for encoder
LE = LabelEncoder()
for col in cols_leave:
    df[col] = LE.fit_transform(df[col])

In [93]:
# convert to single outcome

df["status"].replace({"1": "0", "2": "1"}, inplace=True)
df

,id,time,status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
0,1,400,1,0,58.765229299999994,0,2,2,2,0,14.5,261.0,2.6,156.0,1718.0,137.95,172.0,190.0,12.2,3
1,2,4500,0,0,56.44626968,0,1,2,2,1,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,2
2,3,1012,1,0,70.07255305,1,1,1,1,2,1.4,176.0,3.48,210.0,516.0,96.1,55.0,151.0,12.0,3
3,4,1925,1,0,54.74058864,0,1,2,2,2,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,3
4,5,1504,0,2,38.10540726,0,1,2,2,1,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,414,681,1,1,67.00068446,0,0,0,0,1,1.2,nan,2.96,nan,nan,nan,nan,174.0,10.9,2
413,415,1103,0,1,39.00068446,0,0,0,0,1,0.9,nan,3.83,nan,nan,nan,nan,180.0,11.2,3
414,416,1055,0,1,56.99931554,0,0,0,0,1,1.6,nan,3.42,nan,nan,nan,nan,143.0,9.9,2
415,417,691,0,1,58.00136893,0,0,0,0,1,0.8,nan,3.75,nan,nan,nan,nan,269.0,10.4,2


In [94]:
# split dataset

df_train = df
df_test = df_train.sample(frac=0.2)
df_train = df_train.drop(df_test.index)
df_val = df_train.sample(frac=0.2)
df_train = df_train.drop(df_val.index)


In [95]:
df_train

,id,time,status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
0,1,400,1,0,58.765229299999994,0,2,2,2,0,14.5,261.0,2.6,156.0,1718.0,137.95,172.0,190.0,12.2,3
1,2,4500,0,0,56.44626968,0,1,2,2,1,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,2
2,3,1012,1,0,70.07255305,1,1,1,1,2,1.4,176.0,3.48,210.0,516.0,96.1,55.0,151.0,12.0,3
4,5,1504,0,2,38.10540726,0,1,2,2,1,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,2
6,7,1832,0,2,55.53456537,0,1,2,1,1,1.0,322.0,4.09,52.0,824.0,60.45,213.0,204.0,9.7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,407,1129,0,1,54.00136893,1,0,0,0,1,1.1,nan,3.69,nan,nan,nan,nan,220.0,10.8,2
407,409,1067,0,1,43.00068446,0,0,0,0,1,0.7,nan,3.73,nan,nan,nan,nan,214.0,10.8,2
409,411,1119,0,1,51.00068446,0,0,0,0,1,0.6,nan,3.57,nan,nan,nan,nan,286.0,10.6,2
414,416,1055,0,1,56.99931554,0,0,0,0,1,1.6,nan,3.42,nan,nan,nan,nan,143.0,9.9,2


## Feature transforms

In [96]:
# Standardize the numerical covariates, leave the binary covariates as is
# PyTorch require variables of type 'float32'

cols_standardize = ['age', 'bili', 'chol', 'albumin', 'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime']
cols_leave = ['trt', 'sex', 'ascites', 'hepato', 'spiders', 'edema', 'stage']

standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]

x_mapper = DataFrameMapper(standardize + leave)

In [97]:
x_train = x_mapper.fit_transform(train).astype('float32')
x_val = x_mapper.transform(validate).astype('float32')
x_test = x_mapper.transform(test).astype('float32')

In [109]:
x_train

array([[-0.96334034, -0.23291713,  0.14215931, ...,  1.        ,
         1.        ,  2.        ],
       [ 0.83642703,  5.29025   , -0.00758216, ...,  1.        ,
         1.        ,  1.        ],
       [-0.262098  ,  1.8913778 ,         nan, ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 0.5511849 , -0.4689499 , -0.25421518, ...,  2.        ,
         1.        ,  2.        ],
       [ 1.182303  , -0.5869663 ,         nan, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.7924017 , -0.18571058, -0.19255692, ...,  1.        ,
         1.        ,  1.        ]], dtype=float32)

In [147]:
get_target = lambda df: (df['time'].values, df['status'].values)
y_train = get_target(df_train)
y_val = get_target(df_val)
durations_test, events_test = get_target(df_test)
val = x_val, y_val


In [148]:
y_train

(array(['400', '4500', '1012', '1504', '1832', '2466', '51', '304', '3577',
        '3584', '3672', '131', '4232', '1356', '3445', '4127', '1444',
        '77', '549', '4509', '321', '3839', '4523', '3170', '2847', '3611',
        '223', '3244', '2297', '4467', '1350', '4453', '3428', '2576',
        '2598', '3853', '1000', '1360', '3282', '4459', '4365', '4256',
        '3090', '859', '4191', '2769', '3458', '4184', '4190', '1191',
        '71', '326', '1690', '890', '2540', '3574', '4050', '4032', '3358',
        '1657', '2452', '1741', '460', '3913', '750', '130', '3850',
        '3823', '3820', '552', '3581', '3092', '3222', '2583', '2504',
        '2105', '3445', '980', '3336', '1083', '2288', '3069', '2468',
        '824', '3255', '1037', '3239', '2796', '3150', '3098', '2990',
        '3059', '3050', '2419', '786', '2995', '1427', '2863', '140',
        '2666', '853', '2835', '2475', '2797', '186', '264', '1077',
        '2721', '1682', '1212', '2692', '2574', '2657', '2624', '1

In [153]:
np.asarray([y_train],dtype='<U32')

array([[['400', '4500', '1012', '1504', '1832', '2466', '51', '304',
         '3577', '3584', '3672', '131', '4232', '1356', '3445', '4127',
         '1444', '77', '549', '4509', '321', '3839', '4523', '3170',
         '2847', '3611', '223', '3244', '2297', '4467', '1350', '4453',
         '3428', '2576', '2598', '3853', '1000', '1360', '3282', '4459',
         '4365', '4256', '3090', '859', '4191', '2769', '3458', '4184',
         '4190', '1191', '71', '326', '1690', '890', '2540', '3574',
         '4050', '4032', '3358', '1657', '2452', '1741', '460', '3913',
         '750', '130', '3850', '3823', '3820', '552', '3581', '3092',
         '3222', '2583', '2504', '2105', '3445', '980', '3336', '1083',
         '2288', '3069', '2468', '824', '3255', '1037', '3239', '2796',
         '3150', '3098', '2990', '3059', '3050', '2419', '786', '2995',
         '1427', '2863', '140', '2666', '853', '2835', '2475', '2797',
         '186', '264', '1077', '2721', '1682', '1212', '2692', '2574',
    

In [146]:
y_train[0].astype(np.float)

In [138]:
y_train[1].astype(int)

array([1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0])

# DeepSurv/CoxPH

In [154]:
# create neuralnet

# We create a simple MLP with two hidden layers, ReLU activations, batch norm and dropout.
# Here, we just use the "torchtuples.practical.MLPVanilla" net to do this.
# Note that we set out_features to 1, and that we have not output_bias.

in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = 1
batch_norm = True
dropout = 0.1
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)

In [155]:
# train deepsurv

# To train the model we need to define an optimizer.
# You can choose any torch.optim optimizer, but here we instead use one from tt.optim as it has some added functionality.
# We use the Adam optimizer, but instead of choosing a learning rate, we will use the scheme proposed by Smith 2017 to find a suitable learning rate with model.lr_finder.

model = CoxPH(net, tt.optim.Adam)

In [156]:
# learning rate

batch_size = 256
lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance=10)
_ = lrfinder.plot()

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [126]:
lrfinder.get_best_lr()

NameError: name 'lrfinder' is not defined

# DeepHit

In [68]:
# neural network

in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = labtrans.out_features
batch_norm = True
dropout = 0.1

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)

NameError: name 'labtrans' is not defined

In [ ]:
# train deephit model

model = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)

In [ ]:
# learning rate

batch_size = 256
lr_finder = model.lr_finder(x_train, y_train, batch_size, tolerance=3)
_ = lr_finder.plot()

In [ ]:
lr_finder.get_best_lr()

In [ ]:
# prediction

surv = model.predict_surv_df(x_test)

In [ ]:
# survival estimates for first 5 individuals 

surv.iloc[:, :5].plot(drawstyle='steps-post')
plt.ylabel('S(t | x)')
_ = plt.xlabel('Time')

In [ ]:
# evaluation

ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

In [ ]:
# concordance

ev.concordance_td('antolini')

In [ ]:
# brier score

time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
ev.brier_score(time_grid).plot()
plt.ylabel('Brier score')
_ = plt.xlabel('Time')